In [1]:
!pip install patchify --upgrade
!pip install wandb --upgrade
!pip install segmentation_models_pytorch --upgrade
!pip install warmup_scheduler --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.13.10
    Uninstalling wandb-0.13.10:
      Successfully uninstalled wandb-0.13.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=0e8c2cde1941d11b973cbceb66e02a3617b10093d653dcc18d64359906a93a00
  Stored in directory: /root/.cache/pip/wheels/96/3f/5f/13976445f67f3b4e77b054e65f7f4c39016e92e8358fe088db
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3

In [2]:
# import sys
# sys.path.append('/kaggle/input/smp-vesuvius/EfficientNet-PyTorch-master/EfficientNet-PyTorch-master')
# sys.path.append('/kaggle/input/smp-vesuvius/pretrained-models.pytorch-master/pretrained-models.pytorch-master')
# sys.path.append('/kaggle/input/smp-vesuvius/pytorch-image-models-main/pytorch-image-models-main')
# sys.path.append('/kaggle/input/smp-vesuvius/segmentation_models.pytorch-master/segmentation_models.pytorch-master')

In [3]:
config = {
    'arch':'Unet++',
    'model' :'efficientnet-b5',
    'freeze' :True,
    'weight_decay' : 0.1,
    'in_chans' : 6,
    'version' : 70,
    'batch_size':16,
    'bce': 0,
    'jaccard': 0.5,
    'dice': 0.5,
}

In [4]:
def update_CFG(config,CFG):
    model_name = config['arch']
    backbone = config['model']
    weight_decay = config['weight_decay']
    freeze = config['freeze']
    in_chans = config['in_chans'] 

In [5]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, log_loss, fbeta_score
import sys
import os
import psutil 

import gc
import sys
import math
import random

from tifffile import tifffile
import PIL.Image as Image
from patchify import patchify

import numpy as np
import pandas as pd
import datetime
import time

import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch
import torch.nn as nn
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import wandb
wandb.login(key = '68a8aa0d16f78ed8a8b3f6d0255850bdbf2f7b61')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## config

In [6]:
class CFG:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    size = 224
    step = 112
    valid_id = 1
    in_chans = 6
    batch_size = 16
    num_workers = 2 if torch.cuda.is_available() else 4
    target_size = 1
    train_batch_size = config['batch_size'] # 32
    valid_batch_size = train_batch_size * 2
    use_amp = True if torch.cuda.is_available() else False
    epochs = 15
    warmup_factor = 10
    lr = 1e-4 / warmup_factor
    
    pretrained = True
    inf_weight = 'best'

    min_lr = 1e-7
    max_grad_norm = 1000

    print_freq = 50

    seed = 42
    metric_direction = 'maximize'
    
    # ============== comp exp name =============
    comp_name = 'vesuvius'

    # comp_dir_path = './'
    comp_dir_path = '/kaggle/input/'
    comp_folder_name = 'vesuvius-challenge-ink-detection'
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
    
    exp_name = 'krishkaexp'

    # ============== model cfg =============
    model_name = 'Unet'
    backbone = config['model']
    weight_decay = config['weight_decay']
    freeze = config['freeze']
    in_chans = config['in_chans']

    #scheduler = 'GradualWarmupSchedulerV2'
    # scheduler = 'CosineAnnealingLR'


    # ============== fold =============
    metric_direction = 'maximize'  # maximize, 'minimize'
    # metrics = 'dice_coef'

    # ============== set dataset path =============
    print('set dataset path')

    outputs_path = f'/kaggle/working/outputs/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + 'models/'

    figures_dir = outputs_path + 'figures/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    
    # ============== augmentation =============

set dataset path


## helper

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [8]:
def init_logger(log_file):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 42

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic
    torch.backends.cudnn.benchmark = False

In [9]:
def make_dirs(cfg):
    for dir in [cfg.model_dir, cfg.figures_dir, cfg.submission_dir, cfg.log_dir]:
        os.makedirs(dir, exist_ok=True)

In [10]:
def make_dir(path):
    os.makedirs(path, exist_ok = True)

In [11]:
def cfg_init(cfg, mode='train'):
    set_seed(cfg.seed)
    if mode == 'train':
        make_dirs(cfg)

In [12]:
cfg_init(CFG)

Logger = init_logger(log_file=CFG.log_path)

#Logger.info('\n\n-------- exp_info -----------------')
# Logger.info(datetime.datetime.now().strftime('%Y年%m月%d日 %H:%M:%S'))

## image, mask

In [13]:
path_surface = lambda mode,fragment_id,i: f"/kaggle/input/vesuvius-challenge-ink-detection/{mode}/{fragment_id}/surface_volume/{i:02}.tif"
path_mask = lambda mode,fragment_id : f'/kaggle/input/vesuvius-challenge-ink-detection/{mode}/{fragment_id}/mask.png'
path_label = lambda mode,fragment_id :f'/kaggle/input/vesuvius-challenge-ink-detection/{mode}/{fragment_id}/inklabels.png'

def return_patch_details(fragment_id,CFG,size=256,step = 128):

    dic = {'fragment':[],'xmin':[],'ymin':[],'step':[],'size':[],'%ink':[]}
    mask = np.array(Image.open(path_mask('train',fragment_id)))
    label = np.array(Image.open(path_label('train',fragment_id)))
    
    pad0 = 0
    pad1 = 0
    if mask.shape[0] % step !=0:      
        pad0 = (size - mask.shape[0] % step)
    if mask.shape[1] % step !=0:
        pad1 = (size - mask.shape[1] % step)
    mask = np.pad(mask, [(0, pad0), (0, pad1)], constant_values=0)
    label = np.pad(label, [(0, pad0), (0, pad1)], constant_values=0)
        
    patched_mask = patchify(mask,(size,size),step = step)
    patched_label = patchify(label,(size,size),step = step)
        
    for j in range(patched_mask.shape[0]):
        for k in range(patched_mask.shape[1]):
            mask_p = patched_mask[j,k,:,:]
            if np.count_nonzero(mask_p)==size*size:
                label_p = patched_label[j,k,:,:]
                dic['fragment'].append(fragment_id)
                dic['ymin'].append(j*size)
                dic['xmin'].append(k*size)
                dic['step'].append(step)
                dic['size'].append(size)
                dic['%ink'].append(np.count_nonzero(label_p)*100/(size*size))
    df = pd.DataFrame(dic)
    del mask, label, patched_label, patched_mask, mask_p, label_p
    a = gc.collect()
    return df

In [14]:
def return_image_mask_patch(fragment_id,CFG,thresh):
    
    in_chans = CFG.in_chans
    size = CFG.size
    step = CFG.step
    
    df = return_patch_details(fragment_id,CFG,size=size,step=step)     
    df = df[df['%ink']>=thresh*100]
    images = []

    mid = 65 // 2
    start = mid - in_chans // 2
    end = mid + in_chans // 2 if in_chans%2 == 0 else mid + in_chans // 2 + 1
    
    label_ = np.array(Image.open(path_label('train',fragment_id)))
    
    pad0 = 0
    pad1 = 0
    if label_.shape[0] % step !=0:      
        pad0 = (size - label_.shape[0] % step)
    if label_.shape[1] % step !=0:
        pad1 = (size - label_.shape[1] % step)
    label_ = np.pad(label_, [(0, pad0), (0, pad1)], constant_values=0)
         
    for i in tqdm(range(start,end),desc ="Image Slice Progress",colour = "BLUE"):
        image = tifffile.imread(path_surface('train',fragment_id,i))
        image = np.pad(image, [(0, pad0), (0, pad1)], constant_values=0)
        images.append(image)
        
    images = np.stack(images, axis=2)
    
    image = []
    label = []
    xyxy = []
    
    df = df.to_numpy()
    size = int(df[0,4])
    for j in range(df.shape[0]):
        xmin = int(df[j,1])
        ymin = int(df[j,2])
        img = images[ymin:ymin+size,xmin:xmin+size,:]
        lab = np.expand_dims(label_[ymin:ymin+size,xmin:xmin+size],2)
        if img.shape == (size,size,in_chans) and lab.shape == (size,size,1):
            image.append(img)
            label.append(lab)
            xyxy.append([xmin,ymin,xmin+size,ymin+size])
        
    del img,lab,df,images,label_
    a = gc.collect()
    return image, label, xyxy

In [15]:
def read_all_fragments(CFG,thresh = 0): 
    
    images = []
    labels = [] 
    xyxys = []

    for fragment_id in range(1, 4):
        image, label, xyxy = return_image_mask_patch(fragment_id,CFG, thresh = thresh)
        
        images.append(image)
        labels.append(label)
        xyxys.append(xyxy)
    del image, label, xyxy
    a = gc.collect()
    return images,labels,xyxys

In [16]:
# images,labels,xyxys = read_all_fragments()

## dataset

In [17]:
def get_transforms(data, cfg):
    
    train_aug_list = [
        A.Resize(CFG.size, CFG.size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
                ], p=0.4),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.CoarseDropout(max_holes=1, max_width=int(CFG.size * 0.3), max_height=int(CFG.size * 0.3), 
                        mask_fill_value=0, p=0.5),
        A.Normalize(
            mean= [0] * CFG.in_chans,
            std= [1] * CFG.in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug_list = [
        A.Resize(CFG.size, CFG.size),
        A.Normalize(
            mean= [0] * CFG.in_chans,
            std= [1] * CFG.in_chans
        ),
        ToTensorV2(transpose_mask=True),
    ]
    
    if data == 'train':
        aug = A.Compose(train_aug_list)
    elif data == 'valid':
        aug = A.Compose(valid_aug_list)
    return aug

class CustomDataset(Dataset):
    def __init__(self, images, cfg, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32)/65535
        label = self.labels[idx].astype(np.float32)

        if self.transform:
            data = self.transform(image=image, mask=label)
            image = data['image']
            label = data['mask']

        return image, label

In [18]:
def create_dataloaders_new(CFG,full_images, full_masks, full_xyxys, valid_id = CFG.valid_id):
    valid_id_int = int(valid_id) - 1
    
    valid_images = full_images[valid_id_int]
    valid_masks = full_masks[valid_id_int]
    valid_xyxys  = full_xyxys[valid_id_int]
    
    if valid_id_int == 0:
        train_images = full_images[1] + full_images[2]
        train_masks = full_masks[1] + full_masks[2]
    elif valid_id_int == 1:
        train_images = full_images[0] + full_images[2]
        train_masks = full_masks[0] + full_masks[2] 
    elif valid_id_int == 2:
        train_images = full_images[0] + full_images[1]
        train_masks = full_masks[0] + full_masks[1]  
    
    train_dataset = CustomDataset(
    train_images, CFG, labels=train_masks, transform=get_transforms(data='train', cfg=CFG))
    valid_dataset = CustomDataset(
        valid_images, CFG, labels=valid_masks, transform=get_transforms(data='valid', cfg=CFG))


    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.train_batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                              )
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.valid_batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    return train_loader, valid_loader, valid_xyxys

## model

In [19]:
class CustomModel(nn.Module):
    def __init__(self, cfg, weight=None):
        super().__init__()
        self.cfg = cfg

        self.encoder = smp.PSPNet(
            encoder_name=cfg.backbone, 
            encoder_weights=weight,
            in_channels=cfg.in_chans,
            classes=cfg.target_size,
            activation=None,
        )

    def forward(self, image):
        output = self.encoder(image)
        # output = output.squeeze(-1)
        return output


def build_model(cfg, weight="imagenet"):
    print('model_name', cfg.model_name)
    print('backbone', cfg.backbone)

    model = CustomModel(cfg, weight)
    if CFG.freeze:
        for params in model.encoder.encoder.parameters():
            params.requires_grad = False

    return model

In [20]:
# !pip install torchsummary
# from torchsummary import summary

In [21]:
# summary(model,(6,224,224))

In [22]:
# for params in model.encoder.encoder.parameters():
#     params.requires_grad = False

In [23]:
# summary(model,(6,224,224))

## scheduler

In [24]:
from warmup_scheduler import GradualWarmupScheduler


class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    """
    https://www.kaggle.com/code/underwearfitting/single-fold-training-of-resnet200d-lb0-965
    """
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(
            optimizer, multiplier, total_epoch, after_scheduler)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [
                        base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

def get_scheduler(cfg, optimizer):
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=100, eta_min=1e-7)
    
    #scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min = 1e-6)
    scheduler = GradualWarmupSchedulerV2(
        optimizer, multiplier=10, total_epoch=100, after_scheduler=scheduler_cosine)

    return scheduler

# def scheduler_step(scheduler, avg_val_loss, epoch):
#     scheduler.step(epoch)


In [25]:
def initalize_model(CFG):
    model = build_model(CFG)
    model.to(CFG.device)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay = CFG.weight_decay)
    scheduler = get_scheduler(CFG, optimizer)
    return model, optimizer, scheduler

## loss

In [26]:
# # DiceLoss = smp.losses.DiceLoss(mode='binary')
# BCELoss = smp.losses.SoftBCEWithLogitsLoss()

# # alpha = 0.5
# # beta = 1 - alpha
# # TverskyLoss = smp.losses.TverskyLoss(
# #     mode='binary', log_loss=False, alpha=alpha, beta=beta)

# def criterion(y_pred, y_true):
#     # return 0.5 * BCELoss(y_pred, y_true) + 0.5 * DiceLoss(y_pred, y_true)
#     return BCELoss(y_pred, y_true)
#     # return 0.5 * BCELoss(y_pred, y_true) + 0.5 * TverskyLoss(y_pred, y_true)

In [27]:
Dice = smp.losses.DiceLoss(mode = 'binary',log_loss = True)
Jaccard = smp.losses.JaccardLoss(mode ='binary', log_loss = True)
BCE = smp.losses.SoftBCEWithLogitsLoss()
def criterion(y_pred, y_true):
    criterion = config['dice']*Dice(y_pred,y_true) + config['jaccard']*Jaccard(y_pred,y_true) + config['bce']*BCE(y_pred,y_true)
    return criterion

## train, val

In [28]:
def train_fn(CFG,train_loader, model, criterion, optimizer, device, scheduler):
    model.train()

    scaler = GradScaler(enabled=CFG.use_amp)
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader), desc ="Train Steps Progress",colour = 'RED'):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with autocast(CFG.use_amp):
            y_preds = model(images)
            loss = criterion(y_preds, labels)

        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), CFG.max_grad_norm)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        
    del images, labels
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    a = gc.collect()
    
    return losses.avg

def valid_fn(CFG,valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt):
    mask_pred = np.zeros(valid_mask_gt.shape)
    mask_count = np.zeros(valid_mask_gt.shape)

    model.eval()
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(valid_loader), total=len(valid_loader), desc ="Validation Steps Progress",colour = 'GREEN'):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        # make whole mask
        y_preds = torch.sigmoid(y_preds).to('cpu').numpy().astype(np.float32)
        start_idx = step*CFG.valid_batch_size
        end_idx = start_idx + batch_size
        for i, (x1, y1, x2, y2) in enumerate(valid_xyxys[start_idx:end_idx]):
            mask_pred[y1:y2, x1:x2] += y_preds[i].squeeze(0)
            mask_count[y1:y2, x1:x2] += np.ones((CFG.size, CFG.size),dtype = np.float32)
    
    del images, labels
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    a = gc.collect()
    
    mask_count = np.where(mask_count==0,1,mask_count)
    mask_pred /= mask_count
    
    return losses.avg, mask_pred

## metrics

In [29]:
def fbeta_numpy(targets, preds, beta=0.5, smooth=1e-5):
    """
    https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
    """
    targets = targets.flatten().astype(np.uint8)
    preds = preds.flatten().astype(np.uint8)
    y_true_count = targets.sum()
    ctp = preds[targets==1].sum()
    cfp = preds[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    del targets, preds
    a = gc.collect()
    return dice

def calc_fbeta(mask, mask_pred):
    mask = mask.astype(np.uint8).flatten()
    mask_pred = mask_pred.flatten()

    best_th = 0
    best_dice = 0
    dice = fbeta_numpy(mask, (mask_pred >= 0.5), beta=0.5)
    
    best_dice = dice
    best_th = 0.5 #assumed this is 0.5 vs iterating over different thresholds
# Commented this out to save memory 
    for th in np.array(range(10, 80+1, 5)) / 100:
        
        # dice = fbeta_score(mask, (mask_pred >= th).astype(int), beta=0.5)
        dice = fbeta_numpy(mask, (mask_pred >= th).astype(int), beta=0.5)
        print(f'th: {th}, fbeta: {dice}')

        if dice > best_dice:
            best_dice = dice
            best_th = th
    
    Logger.info(f'best_th: {best_th}, fbeta: {best_dice}')
    del mask, mask_pred
    a = gc.collect()
    return best_dice, best_th

def calc_cv(mask_gt, mask_pred):
    best_dice, best_th = calc_fbeta(mask_gt, mask_pred)

    return best_dice, best_th

## main

In [30]:
def valid_mask_gt_func(CFG,fragment_id = CFG.valid_id):
    size = CFG.size
    step = CFG.step
    
    valid_mask_gt = np.array(Image.open(CFG.comp_dataset_path + f"train/{fragment_id}/inklabels.png"))
    valid_mask_gt = valid_mask_gt
    
    pad0 = 0
    pad1 = 0
    if valid_mask_gt.shape[0] % step !=0:      
        pad0 = (size - valid_mask_gt.shape[0] % step)
    if valid_mask_gt.shape[1] % step !=0:
        pad1 = (size - valid_mask_gt.shape[1] % step)
    
    valid_mask_gt = np.pad(valid_mask_gt, [(0, pad0), (0, pad1)], constant_values=0)
    return valid_mask_gt

In [31]:
def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

In [32]:
cpu_stats()

'memory GB:0.98'

In [33]:
# full_images, full_masks, full_xyxys = read_all_fragments()
# fold = 1
# train_loader, valid_loader, valid_xyxys = create_dataloaders_new(full_images ,full_masks ,full_xyxys, fold)
# valid_mask_gt = valid_mask_gt_func(fold)
# device = CFG.device
# model, optimizer, scheduler = initalize_model()
# vg_val_loss, mask_pre = valid_fn(valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt)

In [34]:
# np.max(mask_pre)

In [35]:
# np.max(valid_mask_gt)

In [36]:
# mask_pre.shape == valid_mask_gt.shape

In [37]:
# valid_mask_gt.dtype

In [38]:
# score = fbeta_numpy(valid_mask_gt,mask_pre>=0.5)

In [39]:
# score

In [40]:
# mask_pre.shape

In [41]:
def train(config,CFG):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    #fold = CFG.valid_id
    print(f'pre inializing: {cpu_stats()}')

    full_images, full_masks, full_xyxys = read_all_fragments(CFG) #reads and puts all fragments in list

    device = CFG.device

    print(f'post inializing: {cpu_stats()}')
    for fold in [1]:

        print(f'Current fold --> {fold}')

        run = wandb.init(project=f"Vesuvius Experiments PSPNet", config=config, name = f"{config['arch']} {config['model']} Fold{fold} freeze{config['freeze']} {random.randint(1,100000)}")

        if CFG.metric_direction == 'minimize':
            best_score = np.inf
        elif CFG.metric_direction == 'maximize':
            best_score = -1

        best_loss = np.inf


        train_loader, valid_loader, valid_xyxys = create_dataloaders_new(CFG,full_images ,full_masks ,full_xyxys, fold) #create_dataloaders(fold)
        valid_mask_gt = valid_mask_gt_func(CFG,fold) #Get validation mask

        model, optimizer, scheduler = initalize_model(CFG)
        _, mask_pred = valid_fn(CFG,valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt)

        best_dice, best_th = calc_cv(valid_mask_gt, mask_pred)
        best_score = best_dice

        print(f'Val_loss before training -->{_}, score --> {best_score}')

        del mask_pred
        gc.collect()

        path = CFG.model_dir + f'fold{fold}/'
        make_dir(path)
        for epoch in tqdm(range(CFG.epochs),desc ="Epoch Progress",colour = 'WHITE'):

            start_time = time.time()

            avg_loss = train_fn(CFG,train_loader, model, criterion, optimizer, device, scheduler)
            avg_val_loss, mask_pred = valid_fn(CFG,valid_loader, model, criterion, device, valid_xyxys, valid_mask_gt)
    #         scheduler_step(scheduler, avg_val_loss, epoch)
            best_dice, best_th = calc_cv(valid_mask_gt, mask_pred)
            score = best_dice

            elapsed = time.time() - start_time

            Logger.info(
                f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
            Logger.info(
                f'Epoch {epoch+1} - avgScore: {score:.4f}')
#             torch.save({'model': model.state_dict(),
#                        'opitmizer': optimizer.state_dict(),
#                         'scheduler':scheduler.state_dict(),},
#                         path + f'{CFG.model_name}_freeze_epoch{epoch}.pth')

            if CFG.metric_direction == 'minimize':
                update_best = score < best_score
            elif CFG.metric_direction == 'maximize':
                update_best = score > best_score

            wandb.log({"Train_loss": avg_loss, "Val_los": avg_val_loss, "Score": score, "Epoch":epoch+1, 'Best_th':best_th})

            if update_best:
                best_loss = avg_val_loss
                best_score = score

                Logger.info(
                    f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
                Logger.info(
                    f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')

                print(f'saving model: {cpu_stats()}')
                torch.save({'model': model.state_dict(),
                           'opitmizer': optimizer.state_dict(),
                           'scheduler':scheduler.state_dict()},
                            path+f'{CFG.model_name}_freeze_best_epoch{epoch}.pth')
                wandb.log({"Best_loss": avg_loss, "Best_Score": best_score, "Epoch_best":epoch+1, 'Best_threshold':best_th})
        del model, valid_mask_gt #Not sure if this is right or not, freeing up memory 

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        abds = gc.collect()
        run.finish()
        print(f'deleted items: {cpu_stats()}')
    del full_images[:], full_masks[:], full_xyxys[:]
    del full_images, full_masks, full_xyxys
    abds = gc.collect()
    return 0

In [42]:
# check_point = torch.load(
#     CFG.model_dir + f'{CFG.model_name}_fold{fold}_{CFG.inf_weight}.pth', map_location=torch.device('cpu'))

In [43]:
# mask_pred = check_point['preds']

In [44]:
# best_dice, best_th  = calc_fbeta(valid_mask_gt, mask_pred)

In [45]:
# fig, axes = plt.subplots(1, 3, figsize=(15, 8))
# axes[0].imshow(valid_mask_gt)
# axes[1].imshow(mask_pred)
# axes[2].imshow((mask_pred>=best_th).astype(int))

In [46]:
# np.unique(mask_pred), np.min(mask_pred), np.max(mask_pred)

In [47]:
# mask_pred

In [48]:
# plt.hist(mask_pred.flatten(), bins=20)

In [49]:
# run.finish()

In [50]:
config = {
    'arch':'PSPNet',
    'model' :'efficientnet-b5',
    'freeze' :True,
    'weight_decay' : 0.1,
    'in_chans' : 6,
    'version' : 79,
    'batch_size':16,
    'bce': 1,
    'jaccard': 0,
    'dice': 0,
}

In [51]:
def update_CFG(config,CFG):
    CFG.model_name = config['arch']
    CFG.backbone = config['model']
    CFG.weight_decay = config['weight_decay']
    CFG.freeze = config['freeze']
    CFG.in_chans = config['in_chans'] 

In [52]:
model_to_try = ['mit_b0','mit_b1','mit_b2','mit_b3','mit_b4','mit_b5','resnet50','efficientnet-b4','efficientnet-b5','efficientnet-b6','timm-mobilenetv3_large_075','mobilenet_v2','efficientnet-b7','timm-mobilenetv3_large_100','timm-mobilenetv3_large_minimal_100']
freeze_try  = [True,False]

for modell in model_to_try:
    config['model'] = modell
        
    if modell in ['mit_b0','mit_b1','mit_b2','mit_b3','mit_b4','mit_b5']:
        config['in_chans'] = 3
    else:
        config['in_chans'] = 6
    update_CFG(config,CFG)
    mod = train(config,CFG)
    abds = gc.collect()

pre inializing: memory GB:0.98


Image Slice Progress: 100%|██████████| 3/3 [00:02<00:00,  1.44it/s]
wandb: Currently logged in as: whybeaking (i-am-noob). Use `wandb login --relogin` to force relogin


post inializing: memory GB:2.55
Current fold --> 1


wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230602_173759-gj4121ru
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run PSPNet mit_b0 Fold1 freezeTrue 76811
wandb: ⭐️ View project at https://wandb.ai/i-am-noob/Vesuvius%20Experiments%20PSPNet
wandb: 🚀 View run at https://wandb.ai/i-am-noob/Vesuvius%20Experiments%20PSPNet/runs/gj4121ru


model_name PSPNet
backbone mit_b0


...

th: 0.1, fbeta: 0.17910150275966857
th: 0.15, fbeta: 0.19118523715415975
th: 0.2, fbeta: 0.15108791469901786
th: 0.25, fbeta: 0.05556799758593655
th: 0.3, fbeta: 0.011790679828911122
th: 0.35, fbeta: 0.0023114034853874816
th: 0.4, fbeta: 0.0004211053982285748
th: 0.45, fbeta: 6.741384687296298e-05
th: 0.5, fbeta: 8.427448004020138e-06
th: 0.55, fbeta: 0.0
th: 0.6, fbeta: 0.0
th: 0.65, fbeta: 0.0
th: 0.7, fbeta: 0.0
th: 0.75, fbeta: 0.0


best_th: 0.15, fbeta: 0.19118523715415975
Epoch 15 - avg_train_loss: 0.3223  avg_val_loss: 0.1659  time: 46s
Epoch 15 - avgScore: 0.1912
Epoch 15 - Save Best Score: 0.1912 Model
Epoch 15 - Save Best Loss: 0.1659 Model


th: 0.8, fbeta: 0.0
saving model: memory GB:7.37


Epoch Progress: 100%|██████████| 15/15 [11:30<00:00, 46.01s/it]
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:     Best_Score ▁▂▂▃▄▅▆███
wandb:      Best_loss █▂▂▂▂▁▂▂▂▁
wandb:        Best_th ▁▁▁▁▁▅▅▅█▅▅▅▁▅▅
wandb: Best_threshold ▁▁▁▁▅▅▅█▅▅
wandb:          Epoch ▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
wandb:     Epoch_best ▁▁▂▃▃▄▅▅▆█
wandb:          Score ▁▂▂▃▂▄▅▆█▆█▇▅▆█
wandb:     Train_loss █▂▂▂▂▂▁▂▂▁▂▁▁▁▁
wandb:        Val_los █▆▅▄▄▂▃▂▄▂▁▂▂▂▁
wandb: 
wandb: Run summary:
wandb:     Best_Score 0.19119
wandb:      Best_loss 0.32234
wandb:        Best_th 0.15
wandb: Best_threshold 0.15
wandb:          Epoch 15
wandb:     Epoch_best 15
wandb:          Score 0.19119
wandb:     Train_loss 0.32234
wandb:        Val_los 0.16587
wandb: 
wandb: 🚀 View run PSPNet timm-mobilenetv3_large_minimal_100 Fold1 freezeTrue 50578 at: https://wandb.ai/i-am-noob/Vesuvius%20Experiments%20PSPNet/runs/6eefcii1
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file

deleted items: memory GB:7.37
